In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings 
warnings.filterwarnings('ignore')
import os 
import re
import FinanceDataReader as fdr
import time

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, roc_auc_score ,accuracy_score, precision_score, recall_score, confusion_matrix
import joblib 

rfc = RandomForestClassifier()
xgb = XGBClassifier()
cat = CatBoostClassifier()
models = [rfc,xgb,cat]

# corp_code

In [3]:
df_kospi = pd.read_csv('./data/recent_kospi_list.csv',index_col=0)
# 코드 반환
def corp_code(corp_name):
    code = df_kospi[df_kospi['Name']==corp_name]['Symbol'].iloc[0]
    code = str(code).zfill(6)
    return code

# stock_price()

In [4]:
# 주가데이터
def stock_price(code,bgn_date= '2016-01-01',end_date= '2022-03-31'):

    df_p = fdr.DataReader(code,bgn_date,end_date)
    df_p = df_p.reset_index()
    df_p = df_p.rename(columns = {'Date': '날짜'})
    df_p = df_p.set_index('날짜',drop=True)
    
    return df_p

# merge()

In [5]:
# merge
def merge(df_count, df_scaled):
#     df_merge = pd.merge(df_count,df_p_scaled['Close_scaled'],left_index=True,right_index=True, how='right')
    df_merge = pd.merge(df_count,df_p['Close'],left_index=True,right_index=True, how='right')
    return df_merge

# add_target()

In [6]:
def add_target(df_merge,window_size, period_rate):

    earning_lst = []
    for i in range(len(df_merge)-window_size):
        
        earning_rate = (df_merge.iloc[i+window_size,-1]- df_merge.iloc[i,-1])/df_merge.iloc[i,-1]

        if earning_rate >= period_rate:
            earning_lst.append(1)
        else:
            earning_lst.append(0)

    df_model = df_merge.iloc[:-window_size,:]
    df_model['target'] = earning_lst
    
    return df_model

In [7]:
code = corp_code('SK')

df_p = stock_price(code)
df_count = pd.read_csv('./data/데이터_뉴스키워드빈도/SK.csv',index_col=0)
df_count.index = pd.DatetimeIndex(df_count.index)
df_merge = merge(df_count,df_p)
df_merge.head()

,차례,핵심기술,부인,공사,개념,대구,장기신용등급,야구,드라마,신한카드,...,사업,억만원,회장,이노베이션,삼성전자,외국,배터리,텔레콤,목표,Close
날짜,,,,,,,,,,,,,,,,,,,,,
2016-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,234500
2016-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,241000
2016-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,239000
2016-01-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,240500
2016-01-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,241500


In [14]:
df_count.columns[-1]

'목표'

In [12]:
df_model = add_target(df_merge,30,0.1)

In [13]:
df_model

,차례,핵심기술,부인,공사,개념,대구,장기신용등급,야구,드라마,신한카드,...,억만원,회장,이노베이션,삼성전자,외국,배터리,텔레콤,목표,Close,target
날짜,,,,,,,,,,,,,,,,,,,,,
2016-01-04,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,234500,0
2016-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,241000,0
2016-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,239000,0
2016-01-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,240500,0
2016-01-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,241500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,2,6,223000,0
2022-02-10,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,2,1,12,8,225500,0
2022-02-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,3,223000,1


In [7]:
window_size = 30
period_rate= 0.1

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:] :
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc[:'2021-10-01',:last_col]
        y_train = df_model.loc[:'2021-10-01','target']

        x_test = df_model.loc['2021-10-01':,:last_col]
        y_test = df_model.loc['2021-10-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            if model_name == 'CatBoostClassifier':
                model.fit(x_train,y_train, silent=True)
            else:
                model.fit(x_train,y_train)

            pred = model.predict(x_test)
            accuracy = accuracy_score(y_test, pred)
            proba = model.predict_proba(x_test)[:,1]
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)

            try:
                roc_auc = roc_auc_score(y_test, proba)
            except:
                roc_auc = 0
                
            print(classification_report(y_test,pred))

            if (precision > 0.6) & (recall > 0.4) & (precision!=1):
                save_path = f'./data/machine_model2_{window_size}일_{period_rate}/{corp_name}_{model_name}.pkl'
                joblib.dump(model, save_path)
                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
#                 get_clf_eval(y_test,pred, proba)
                print(save_path)
                print('-'*50)
    
    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result/machine_model2_{window_size}일_{period_rate}.csv')


회사이름:  AJ네트웍스
              precision    recall  f1-score   support

           0       0.83      0.99      0.90        75
           1       0.67      0.12      0.20        17

    accuracy                           0.83        92
   macro avg       0.75      0.55      0.55        92
weighted avg       0.80      0.83      0.77        92

[18:26:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.82      1.00      0.90        75
           1       1.00      0.06      0.11        17

    accuracy                           0.83        92
   macro avg       0.91      0.53      0.51        92
weighted avg       0.86      0.83      0.76        92

              precisio

[18:27:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.92      0.93      0.92        85
           1       0.00      0.00      0.00         7

    accuracy                           0.86        92
   macro avg       0.46      0.46      0.46        92
weighted avg       0.85      0.86      0.85        92

              precision    recall  f1-score   support

           0       0.92      0.89      0.90        85
           1       0.00      0.00      0.00         7

    accuracy                           0.83        92
   macro avg       0.46      0.45      0.45        92
weighted avg       0.85      0.83      0.84        92

회사이름:  CJ제일제당
              precisio

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        88
           1       0.25      0.25      0.25         4

    accuracy                           0.93        92
   macro avg       0.61      0.61      0.61        92
weighted avg       0.93      0.93      0.93        92

회사이름:  DRB동일
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        92

    accuracy                           1.00        92
   macro avg       1.00      1.00      1.00        92
weighted avg       1.00      1.00      1.00        92

[18:27:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.0

              precision    recall  f1-score   support

           0       1.00      0.99      0.99        92
           1       0.00      0.00      0.00         0

    accuracy                           0.99        92
   macro avg       0.50      0.49      0.50        92
weighted avg       1.00      0.99      0.99        92

[18:27:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        92

    accuracy                           1.00        92
   macro avg       1.00      1.00      1.00        92
weighted avg       1.00      1.00      1.00        92

              precision    recall  f1-score   support

           0       1.00      1.00  

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

...

[18:48:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        84
           1       0.00      0.00      0.00         8

    accuracy                           0.91        92
   macro avg       0.46      0.50      0.48        92
weighted avg       0.83      0.91      0.87        92

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        84
           1       0.00      0.00      0.00         8

    accuracy                           0.91        92
   macro avg       0.46      0.50      0.48        92
weighted avg       0.83      0.91      0.87        92

회사이름:  현대로템
              precision 

              precision    recall  f1-score   support

           0       0.82      0.99      0.90        76
           1       0.00      0.00      0.00        16

    accuracy                           0.82        92
   macro avg       0.41      0.49      0.45        92
weighted avg       0.68      0.82      0.74        92

회사이름:  현대엘리베이
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        92
           1       0.00      0.00      0.00         0

    accuracy                           0.99        92
   macro avg       0.50      0.49      0.50        92
weighted avg       1.00      0.99      0.99        92

[18:48:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precisio

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        90
           1       0.00      0.00      0.00         2

    accuracy                           0.98        92
   macro avg       0.49      0.50      0.49        92
weighted avg       0.96      0.98      0.97        92

[18:48:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        90
           1       0.00      0.00      0.00         2

    accuracy                           0.98        92
   macro avg       0.49      0.50      0.49        92
weighted avg       0.96      0.98      0.97        92

              precision    recall  f

[18:49:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.95      0.81      0.88        86
           1       0.11      0.33      0.17         6

    accuracy                           0.78        92
   macro avg       0.53      0.57      0.52        92
weighted avg       0.89      0.78      0.83        92

              precision    recall  f1-score   support

           0       0.93      0.87      0.90        86
           1       0.00      0.00      0.00         6

    accuracy                           0.82        92
   macro avg       0.46      0.44      0.45        92
weighted avg       0.87      0.82      0.84        92

회사이름:  화천기공
              precision 

              precision    recall  f1-score   support

           0       0.93      0.99      0.96        86
           1       0.00      0.00      0.00         6

    accuracy                           0.92        92
   macro avg       0.47      0.49      0.48        92
weighted avg       0.87      0.92      0.90        92

회사이름:  휴스틸
              precision    recall  f1-score   support

           0       0.63      1.00      0.77        58
           1       0.00      0.00      0.00        34

    accuracy                           0.63        92
   macro avg       0.32      0.50      0.39        92
weighted avg       0.40      0.63      0.49        92

[18:49:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision  

In [10]:
x_test

,케미컬탱커,영업익,법정관리,반사이익,가능성,장학금,국내,유지,하락세,지분,...,팬오션,대한해운,하락,현대상선,와이엔텍,통합,해운,해운사,상승률,상승
날짜,,,,,,,,,,,,,,,,,,,,,
2021-10-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [8]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소


In [23]:
# 모델 검증하기
# 한진칼,  
corp_name = '대웅'
model_path = './data/machine_model2_30일_0.1/대웅_CatBoostClassifier.pkl'
window_size = 30
period_rate= 0.1

code = corp_code(corp_name)
df_p = stock_price(code)

path = './data/데이터_뉴스키워드빈도/'
file_path = os.path.join(path,corp_name+'.csv')
df_count = pd.read_csv(file_path,index_col=0)
df_count.index = pd.DatetimeIndex(df_count.index)

df_merge = merge(df_count,df_p)
df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

x_train = df_model.loc[:'2021-10-01',:'Close']
y_train = df_model.loc[:'2021-10-01','target']

x_test = df_model.loc['2021-10-01':,:'Close']
y_test = df_model.loc['2021-10-01':,'target']


# featurn importance
model = joblib.load(model_path)
pred = model.predict(x_test)

print(classification_report(y_test,pred))


              precision    recall  f1-score   support

           0       0.92      0.93      0.93        74
           1       0.71      0.67      0.69        18

    accuracy                           0.88        92
   macro avg       0.81      0.80      0.81        92
weighted avg       0.88      0.88      0.88        92



In [24]:
x_test

,중동,협력,보툴리눔톡신,오송공장,자회사,상대,순항,클럽,수익성,올해,...,임상,신약,시각,상승,외국,메디톡스,치료제,보톡스,나보타,Close
날짜,,,,,,,,,,,,,,,,,,,,,
2021-10-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,32400
2021-10-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30250
2021-10-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,29250
2021-10-07,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,2,0,1,0,32050
2021-10-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,31950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27100
2022-02-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27500
2022-02-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26750


In [26]:
pred

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1], dtype=int64)

In [29]:
idx = np.where(y_test[y_test==pred])

In [31]:
x_test.iloc[idx]

,중동,협력,보툴리눔톡신,오송공장,자회사,상대,순항,클럽,수익성,올해,...,임상,신약,시각,상승,외국,메디톡스,치료제,보톡스,나보타,Close
날짜,,,,,,,,,,,,,,,,,,,,,
2022-01-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,29550
2022-01-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30100
2022-01-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,29600
2022-01-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33150
2022-01-17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30400
2022-01-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,28150
2022-01-19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27400
2022-01-20,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,27200
2022-01-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27050
